# Beachwatch

This notebook examines the bacteria count data for the San Diego coastline, from the Beachwatch program. To analyze beachwatch data, we'll use the data package that is stored on the Library's data repository.

First, visit the [repository home page](http://data.sandiegodata.org) and note the tag for "water-project" below the search box. The [water-project](https://data.sandiegodata.org/dataset?tags=water-project) tag page lists all of the datasets for this project. In the (San Diego Beachwatch Data)[https://data.sandiegodata.org/dataset/ceden-waterboards-ca-gov-beachwatch-sandiego), Look for these two headings, just above the "Data and Resources" section:

- Loading the ZIP Package
- Loading the CSV Package

You can copy the code from one of those sections to get started. 

After opening the data package, we'll look at the results for the stations and station groups, a examine how well readons at one station are correlate with others in the same group. 


## Useful sites

* A great example of [mapping with geopandas](http://jonathansoma.com/lede/foundations-2017/classes/geopandas/mapping-with-geopandas/).


In [53]:
import matplotlib.pyplot as plt 
import metapack as mp
import pandas as pd
import numpy as np
import seaborn as sns

# Get the Package

Usually, the first thing you'll do with a Metatab data package is display the top level documentation, to see what resources it has and other basic information


In [2]:
pkg = mp.open_package('http://library.metatab.org/ceden.waterboards.ca.gov-beachwatch-sandiego-2.zip')

pkg

The Beachwatch dataset has a lot of column, which  we haven't fully documented, so you'll have to guess at what a lot of the column mean. 

In [3]:
# Displaying a resource gives you the schema. This one isn't complete, since we havent filled in the column descriptions. 
pkg.resource('beachwatch-sd')

Header,Type,Description
program,string,
parentproject,string,
project,string,
stationname,string,
stationcode,string,
sampledate,date,
collectiontime,time,
locationcode,string,
collectiondepth,integer,
unitcollectiondepth,string,


# Open the Resource

Below is another really common pattern. Get the resource and extract a Pandas DataFrame, using read_csv(). (You can also use ``.dataframe()``, which has more accurate datatypes, but is slower. ) We'll do some column modifications immediately, then display the data. 

In [4]:
df = pkg.resource('beachwatch-sd').read_csv(parse_dates=True)

# It looks like the prefix of the station code groups stations, maybe into watersheds. 
df['stationgroup'] = df.stationcode.str[:2]

# The results has a large range, so log transformation makes them easier to visualize.
df['log_result'] = df.result.apply(np.log10)

df.head()

,program,parentproject,project,stationname,stationcode,sampledate,collectiontime,locationcode,collectiondepth,unitcollectiondepth,...,huc8,huc8_number,huc10,huc10_number,huc12,huc12_number,waterbody_type,sampleid,stationgroup,log_result
0,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"EH-010-Imperial Beach municipal beach, other, ...",EH-010,1999-04-02,2018-08-03,SurfZone,-88,NR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Recorded,EH,2.477121
1,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"EH-010-Imperial Beach municipal beach, other, ...",EH-010,1999-04-29,2018-08-03,SurfZone,-88,NR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Recorded,EH,1.301030
2,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"EH-010-Imperial Beach municipal beach, other, ...",EH-010,1999-04-30,2018-08-03,SurfZone,-88,NR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Recorded,EH,1.301030
3,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"EH-010-Imperial Beach municipal beach, other, ...",EH-010,1999-05-26,2018-08-03,SurfZone,-88,NR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Recorded,EH,1.000000
4,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"EH-010-Imperial Beach municipal beach, other, ...",EH-010,1999-05-26,2018-08-03,SurfZone,-88,NR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Recorded,EH,1.301030


The different station group have differnt numbers of stations. 

In [5]:
df[['stationcode','stationgroup']].drop_duplicates().groupby('stationgroup').count()

,stationcode
stationgroup,
EH,71
EN,5
FM,10
IB,8
MB,46
OC,11
PL,10
SC,2
SE,7


In [6]:
df['stationcode'].value_counts().head()

IB-080    5065
EH-420    4417
FM-010    4150
SE-060    3780
OC-100    3614
Name: stationcode, dtype: int64

The dataset was extracted with only bacteria counts, but there are several different types. 

In [7]:
df['analyte'].value_counts().head()

Coliform, Total    66992
Coliform, Fecal    65827
Enterococcus       65010
E. coli             4428
Name: analyte, dtype: int64

There are also several difernt analysis methods, with different units. 

In [8]:
df['unit'].value_counts().head()


MPN/100 mL    143346
cfu/100mL      58911
Name: unit, dtype: int64

In [9]:
df['methodname'].value_counts().head()

SM 9222 B     49791
MTF           43466
Enterolert    42372
SM 9221 E     28924
SM 9221 B     25578
Name: methodname, dtype: int64

To ensure that the following comparisions make sense, we'll want to focus on just one type of bacteria count

In [10]:
dfs = df[(df.analyte == 'Coliform, Total') & (df.unit == 'MPN/100 mL')]
len(dfs)

47515